In [14]:
import sys
!{sys.executable} -m pip install webdriver_manager

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

In [1]:
import pandas as pd

In [2]:
Bcorp = pd.read_csv(r"C:\Users\engele016\OneDrive - Gemeente Amsterdam\Documenten\Github\Impact-data-scraper\B Corp Impact Data.csv")

C:\Users\engele016\AppData\Local\Temp\ipykernel_34392\1910107611.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Bcorp = pd.read_csv(r"C:\Users\engele016\OneDrive - Gemeente Amsterdam\Documenten\Github\Impact-data-scraper\B Corp Impact Data.csv")


In [3]:
Bcorp = Bcorp[["company_name","date_first_certified", "date_certified", "current_status","website", "b_corp_profile", "sector", "city", "country", "other_countries_of_operation" ]]

In [4]:
filtered = Bcorp[
    (Bcorp["city"] == "Amsterdam") |
    (Bcorp["other_countries_of_operation"] == "The Netherlands")|
    (Bcorp["city"] == "AMSTERDAM")]

In [5]:
filtered = filtered.drop_duplicates(subset="company_name", keep="first")

In [ ]:
import re
import time
import random
import json
from urllib.parse import urljoin

import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.Firefox import FirefoxDriverManager


POSTAL_RE = re.compile(r"\b(\d{4})\s?([A-Z]{2})\b", re.IGNORECASE)


def normalize_url(url: str) -> str:
    url = str(url).strip()
    if not url:
        return url
    if not url.startswith(("http://", "https://")):
        url = "https://" + url
    return url


def wait_ready(driver, timeout=15):
    WebDriverWait(driver, timeout).until(lambda d: d.execute_script("return document.readyState") == "complete")


def safe_click(driver, element) -> bool:
    try:
        driver.execute_script("arguments[0].click();", element)
        return True
    except Exception:
        try:
            element.click()
            return True
        except Exception:
            return False


def dismiss_common_cookie_banners(driver):
    """
    Best-effort: tries known IDs + common button texts (Dutch/English) + some modal close buttons.
    Also attempts in iframes (many CMPs render there).
    """
    known_selectors = [
        # OneTrust
        (By.ID, "onetrust-accept-btn-handler"),
        (By.CSS_SELECTOR, "button#onetrust-accept-btn-handler"),
        (By.CSS_SELECTOR, "button[aria-label='Accept cookies']"),
        # Cookiebot
        (By.ID, "CybotCookiebotDialogBodyLevelButtonAccept"),
        (By.ID, "CybotCookiebotDialogBodyButtonAccept"),
        (By.CSS_SELECTOR, "#CybotCookiebotDialogBodyLevelButtonAccept"),
        # Complianz
        (By.CSS_SELECTOR, ".cmplz-accept"),
        (By.CSS_SELECTOR, ".cmplz-btn.cmplz-accept"),
        # Generic
        (By.CSS_SELECTOR, "button[aria-label*='accept' i]"),
        (By.CSS_SELECTOR, "button[title*='accept' i]"),
    ]

    button_texts = [
        "accepteer", "alles accepteren", "akkoord", "ik ga akkoord", "toestaan", "prima", "ok",
        "accept", "accept all", "agree", "allow all", "got it", "continue",
    ]

    close_texts = ["sluiten", "close", "×", "x"]

    def try_in_current_context():
        # 1) Try known selectors
        for by, sel in known_selectors:
            try:
                els = driver.find_elements(by, sel)
                for el in els:
                    if el.is_displayed():
                        if safe_click(driver, el):
                            time.sleep(0.3)
                            return True
            except Exception:
                pass

        # 2) Try buttons/links/divs with common consent text
        # Use translate() to lowercase in XPath
        for t in button_texts:
            xpath = (
                "//*[self::button or self::a or @role='button' or self::div]"
                f"[contains(translate(normalize-space(.), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{t}')]"
            )
            try:
                els = driver.find_elements(By.XPATH, xpath)
                for el in els[:6]:
                    if el.is_displayed():
                        if safe_click(driver, el):
                            time.sleep(0.3)
                            return True
            except Exception:
                pass

        # 3) Try generic close buttons on overlays/modals
        for t in close_texts:
            xpath = (
                "//*[self::button or self::a or @role='button']"
                f"[contains(translate(normalize-space(.), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{t}')]"
            )
            try:
                els = driver.find_elements(By.XPATH, xpath)
                for el in els[:6]:
                    if el.is_displayed():
                        if safe_click(driver, el):
                            time.sleep(0.2)
                            return True
            except Exception:
                pass

        return False

    # Try a few rounds, including iframes
    for _ in range(4):
        if try_in_current_context():
            continue

        # Try inside iframes (CMPs often live there)
        try:
            iframes = driver.find_elements(By.TAG_NAME, "iframe")
        except Exception:
            iframes = []

        switched = False
        for fr in iframes[:10]:
            try:
                driver.switch_to.frame(fr)
                switched = True
                if try_in_current_context():
                    driver.switch_to.default_content()
                    break
                driver.switch_to.default_content()
            except Exception:
                try:
                    driver.switch_to.default_content()
                except Exception:
                    pass

        if not switched:
            break


def extract_address_from_html(html: str) -> str | None:
    soup = BeautifulSoup(html, "lxml")

    # 1) Prefer JSON-LD structured data (Organization / LocalBusiness)
    for script in soup.select("script[type='application/ld+json']"):
        try:
            data = json.loads(script.get_text(strip=True))
        except Exception:
            continue

        # JSON-LD can be dict or list
        nodes = data if isinstance(data, list) else [data]
        for node in nodes:
            if not isinstance(node, dict):
                continue

            # Sometimes wrapped in @graph
            graph = node.get("@graph")
            if isinstance(graph, list):
                nodes2 = graph
            else:
                nodes2 = [node]

            for n in nodes2:
                if not isinstance(n, dict):
                    continue
                addr = n.get("address")
                if isinstance(addr, dict):
                    street = addr.get("streetAddress") or ""
                    postal = addr.get("postalCode") or ""
                    locality = addr.get("addressLocality") or ""
                    region = addr.get("addressRegion") or ""
                    country = addr.get("addressCountry") or ""

                    # Require Dutch postal OR Amsterdam-ish signal
                    candidate = " ".join(x for x in [street, postal, locality, region, country] if x).strip()
                    if not candidate:
                        continue

                    has_postal = bool(POSTAL_RE.search(candidate))
                    has_ams = "amsterdam" in candidate.lower() or " ams" in candidate.lower()

                    if has_postal and (has_ams or locality == "" or locality is None):
                        # enforce "Amsterdam" present
                        if "amsterdam" not in candidate.lower():
                            candidate = candidate + " Amsterdam"
                        return clean_address(candidate)

    # 2) Fallback: scan visible text for postal code + Amsterdam
    text_lines = [ln.strip() for ln in soup.get_text("\n").splitlines() if ln.strip()]
    candidates = []

    for i, ln in enumerate(text_lines):
        if POSTAL_RE.search(ln):
            window = " ".join(text_lines[max(0, i - 2): min(len(text_lines), i + 3)])
            if "amsterdam" in window.lower():
                candidates.append(window)

    if candidates:
        # pick shortest reasonable candidate
        candidates.sort(key=len)
        return clean_address(candidates[0])

    # 3) If postal code but Amsterdam missing, still accept and append Amsterdam
    for i, ln in enumerate(text_lines):
        if POSTAL_RE.search(ln):
            window = " ".join(text_lines[max(0, i - 2): min(len(text_lines), i + 3)])
            cand = clean_address(window)
            if "amsterdam" not in cand.lower():
                cand = cand + " Amsterdam"
            return cand

    return None


def clean_address(s: str) -> str:
    # Normalize whitespace and ensure postal code spacing like "1011 DC"
    s = re.sub(r"\s+", " ", s).strip()
    s = re.sub(r"\b(\d{4})\s*([A-Z]{2})\b", r"\1 \2", s, flags=re.IGNORECASE)
    return s


def scrape_address_for_site(driver, url: str) -> str | None:
    url = normalize_url(url)
    if not url:
        return None

    def load_and_extract(u: str) -> str | None:
        driver.get(u)
        wait_ready(driver, timeout=20)
        time.sleep(0.7)
        dismiss_common_cookie_banners(driver)

        # Scroll a bit (addresses often in footer)
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.8)
            dismiss_common_cookie_banners(driver)

        addr = extract_address_from_html(driver.page_source)
        if addr and "amsterdam" in addr.lower() and POSTAL_RE.search(addr):
            return addr

        # If extraction returns something but missing "Amsterdam", enforce it
        if addr and POSTAL_RE.search(addr):
            if "amsterdam" not in addr.lower():
                addr = addr + " Amsterdam"
            return addr

        return None

    # 1) Homepage
    addr = load_and_extract(url)
    if addr:
        return addr

    # 2) Common contact paths
    contact_paths = [
        "/contact", "/contact/", "/contact-us", "/contactgegevens", "/contacteer-ons",
        "/over-ons", "/locatie", "/vestiging", "/waar-zijn-wij", "/route",
    ]
    for path in contact_paths:
        try:
            addr = load_and_extract(urljoin(url, path))
            if addr:
                return addr
        except Exception:
            continue

    return None


def build_driver(headless: bool = False) -> webdriver.Chrome:
    opts = webdriver.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,900")
    opts.add_argument("--disable-notifications")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")

    # Make automation a bit less obvious (not magic, but helps)
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    driver.set_page_load_timeout(40)
    return driver


# --------- MAIN: apply to your dataframe ---------

# filtered must have a column: 'website'
def add_addresses(filtered: pd.DataFrame, headless: bool = False) -> pd.DataFrame:
    out = filtered.copy()
    out["address"] = None

    driver = build_driver(headless=headless)
    try:
        for idx, site in out["website"].items():
            try:
                time.sleep(random.uniform(0.8, 1.6))  # be polite
                addr = scrape_address_for_site(driver, site)
                out.at[idx, "address"] = addr
                print(f"[{idx}] {site} -> {addr}")
            except Exception as e:
                out.at[idx, "address"] = None
                print(f"[{idx}] {site} -> ERROR: {e}")
    finally:
        driver.quit()

    return out
